In [ ]:
library(readxl)
install.packages("gsheet")
library(tidyverse)
library(repr)
library(digest)
library(infer)
library(cowplot)
library(broom)
library(GGally)
library(AER)
library(gsheet)
library(gridExtra)
library(leaps)
library(glmnet)
library(mltools)

In [ ]:
# set seed for notebook
set.seed(7777)

# How Spotify Song Attributes Affect Valence

## Introduction


Starting from infancy and all throughout one's life, listening to music is an activity that the majority of people engage in. Music has been shown to have effects on our daily functioning and overall well-being (Groarke & Hogan, 2018). In particular, how positive or how negative a song sounds plays a critical role in determining how someone will be affected by listening to it. For example, listening to positive sounding music increases self-confidence (Tajadura-Jiménez, 2011) and can help relieve pain (Roy, Peretz, & Rainville, 2008). This musical positiveness present in a song is referred to as its valence, so a song that sounds cheerful and upbeat is said to have high valence (close to 1 in our case). 

Music today is becoming more and more accessible as people can choose what they listen to on the radio, youtube, or on one of the many available streaming services. Currently, Spotify is the world’s most popular music streaming service with over 365 million monthly users (2021). We will be using a dataset containing 232725 songs available on Spotify that contains information about each song such as its genre, danceability, loudness, acousticness, and valence (among others). Our goals are to predict the valence of a song from its other attributes and determine what qualities of a song impact our emotional perception. Therefore, our project will focus on both inference and prediction. First, we will determine which features of the song are most related to its valence, and using these features as our explanatory variables, we will try to build a model to predict a song’s valence.





## Methods & Results

### Preliminary Data Analysis

In [ ]:
# Read in the data and drop any na values in the dataset.
df <- gsheet2tbl('https://docs.google.com/spreadsheets/d/1fnNjFKFZmGgavOYezz-swtQICqWyAMMZxs2H6Nsk29M/edit#gid=2110346216')
df = drop_na(df)
print("Table 1: Spotify Dataset")
head(df)


In [ ]:
# Select relevant features from the dataset, removing metadata such as artist_name, track_name etc.
spotify = df %>% select(-artist_name,  -track_name, -track_id, -duration_ms)
print("Table 2: Spotify Dataset without metadata")
head(spotify)

In [ ]:
# Visualise the distribution of valence across all genres
# Note that the response variable is on the x-axis, this was done because due to the amount of different genres, having genre on the x axis
# made the labels difficult to read, even with rotated text.
options(repr.plot.width = 10, repr.plot.height = 11)
genre_plot = df %>%
ggplot() +
geom_boxplot(aes(y = genre, x = valence)) + 
labs(x = "Valence of Songs", y = "Song Genres", title = "Figure 1: Valence distribution in songs by Genre")

genre_plot



We have a wide variety of distributions for valence over different levels of genre, so it might be worthwhile to explore how the relationship between our predictive variables and the response is affected by genre

In [ ]:
# Visualise the distribution of valence among different levels of categorical features, note that the response variable(valence) is plotted on the y axis from here onwards
options(repr.plot.width = 18, repr.plot.height = 8)
mode_plot = spotify %>%
ggplot() +
geom_boxplot(aes(y = valence, x = mode)) + 
labs(x = "Mode of Songs", y = "Valence of Songs", title = "Figure 2: Valence distribution in songs by Mode")

key_plot = spotify %>%
ggplot() +
geom_boxplot(aes(y = valence, x = key)) + 
labs(x = "Key of Songs", y = "Valence of Songs", title = "Figure 3: Valence distribution in songs by Key")

t_plot = spotify %>%
ggplot() +
geom_boxplot(aes(y = valence, x = time_signature)) + 
labs(x = "Time Signature of Songs", y = "Valence of Songs", title = "Figure 4: Valence distribution in songs by Time Signature")


 plot_grid(mode_plot, key_plot, t_plot,
   ncol = 3
 )


The distribution of valence seems almost identical across different modes, varies slightly across different keys, and varies moderately across different time signatures.

In [ ]:
# Visualise the correlation between valence and numeric variables
# Note that running ggpairs with all the numeric variables was causing the kernel to timeout, so a random subsample of 1000 was used for visualization purposes
options(repr.plot.width = 25, repr.plot.height = 18)
pairs_plot = spotify %>%
            select(-genre, -key, -mode, -time_signature) %>%
            sample_n(size = 1000, replace = FALSE) %>%
            ggpairs(lower=list(combo=wrap("facethist", binwidth=0.8))) + ggtitle("Figure 5: Correlation Between Different Variables")

pairs_plot

Energy and loudness appear to be highly positively correlated (0.826) compared to the rest of the variables.

In [ ]:
# Creating a correlation matrix to examine the relationship between predictive variables
corr_matrix_sample <- spotify %>%
  select(-genre, -key, -mode, -time_signature) %>%
  cor() %>%
  as.data.frame() %>%
  rownames_to_column("var1") %>%
  pivot_longer(-var1, names_to = "var2", values_to = "corr")

plot_corr_matrix_sample <- corr_matrix_sample %>%
  ggplot(aes(x = var1, y = var2)) +
  geom_tile(aes(fill = corr), color = "white") +
  scale_fill_distiller("Correlation Coefficient \n",
    palette = 'YlOrRd',
    direction = 1, limits = c(-1, 1)
  ) +
  labs(x = "", y = "", title = "Figure 6. Correlational Matrix Heatmap") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(
      angle = 45, vjust = 1,
      size = 18, hjust = 1
    ),
    axis.text.y = element_text(
      vjust = 1,
      size = 18, hjust = 1
    ),
    legend.title = element_text(size = 18, face = "bold"),
    legend.text = element_text(size = 18),
    legend.key.size = unit(2, "cm")
  ) +
  coord_fixed() +
  geom_text(aes(x = var1, y = var2, label = round(corr, 2)), color = "black", size = 6)
plot_corr_matrix_sample

To determine if loudness and energy may be contributing to multicollinearity, we will first perform feature selection to see if they are relevant for predicting valence. Then, based on the Variance Inflation Factors, we will assess which model may be most effective for predicting new song valences.

### Variable Selection

In [ ]:
# Removing categorical variables in the dataset, while it would have been nice to include these, our kernel 
# would either end up dying when we tried to fit our models or the output would be too complex to interpret due to the 
# amount of levels they introduced.
spotify <- spotify %>% select(-genre, -key, -mode, -time_signature)

# Splitting the dataset to avoid bias
spotify_split <- sample(rep(1:3, diff(floor(nrow(spotify) * c(0, 0.6, 0.8, 1)))))

# For the model
inference <- spotify[spotify_split==1,]

# For variable selection
selection <- spotify[spotify_split==2,]

# For testing our model
prediction_df <- spotify[spotify_split==3,]

In [ ]:
print("Table 3: Dataset for Feature Selection")
head(selection)

In [ ]:
#Run feature selection, using both forward and backward selection
spotify_sample_forward <- regsubsets(
  x = valence ~.,
  data = selection, nvmax = 50, method = "forward",
)

fwd_summary <- summary(spotify_sample_forward)

fwd_summary_df <- data.frame(
    RSQ = fwd_summary$rsq,
    RSS = fwd_summary$rss,
    ADJ.R2 = fwd_summary$adjr2,
    BIC = fwd_summary$bic
)

spotify_sample_backward <- regsubsets(
  x = valence ~.,
  data = selection, nvmax = 50, method = "backward",
)


back_summary <- summary(spotify_sample_backward)

back_summary_df <- data.frame(
    RSQ = back_summary$rsq,
    RSS = back_summary$rss,
    ADJ.R2 = back_summary$adjr2,
    BIC = back_summary$bic
)
print("Table 4: Summary of Models with different number of input variables selected by forward selection")
fwd_summary_df
print("Table 5: Summary of Models with different number of input variables selected by backward selection")
back_summary_df

In [ ]:
# Making scatter graphs to compare mallow's cp for the different models, so that we can pick the one with least apparent bias.
fwd_candidates <- tibble(
  n_input_variables = 1:9,
  C_p = fwd_summary$cp
)
fwd_plot = fwd_candidates %>%
            ggplot() +
            geom_point(aes(y = C_p, x = n_input_variables)) +
            labs(x = "Number of Input Variables",
                y = "Mallows' CP", 
                title = "Figure 7: Mallows' CP for Models with different number of input variables selected by forward selection")


back_candidates <- tibble(
  n_input_variables = 1:9,
  C_p = back_summary$cp
)
back_plot =  back_candidates %>%
            ggplot() +
            geom_point(aes(y = C_p, x = n_input_variables)) +
            labs(x = "Number of Input Variables",
                y = "Mallows' CP",
                title = "Figure 8: Mallows' CP Models with different number of input variables selected by backward selection")

 plot_grid(fwd_plot, back_plot,
   ncol = 2
 )

Looking at these values, we can see that Mallow's cp is lowest with all the predictors whether we do forward or backward selection, thus we will not remove any variables from our model.

### Variable Selection using Lasso

In [ ]:
# Running Feature selection using Lasso
lasso_model <-
    cv.glmnet(x = selection %>% select(-valence) %>% as.matrix(), 
              y = selection %>% select(valence) %>% as.matrix(), 
              alpha = 1)
plot(lasso_model, main = "Figure 9. Lambda selection by CV with LASSO")

### Feature Selection using Ridge

In [ ]:
# Running Feature selection using Ridge
ridge_model <-
    cv.glmnet(x = selection %>% select(-valence) %>% as.matrix(), 
              y = selection %>% select(valence) %>% as.matrix(), 
              alpha = 0)
plot(ridge_model, main = "Figure 10. Lambda selection by CV with Ridge")

In [ ]:
# Full model via ordinary least squares
full_model <- lm(valence ~ ., data = selection)

# LASSO model with lambda producing lowest MSE
min_lasso_model <- glmnet(
    x = selection %>% select(-valence) %>% as.matrix(),
    y = selection %>% select(valence) %>% as.matrix(),
    alpha = 1,
    lambda = lasso_model$lambda.min)

min_lasso_model.coef <- min_lasso_model$beta

# Ridge model with lambda producing lowest MSE
min_ridge_model <- glmnet(
    x = selection %>% select(-valence) %>% as.matrix(),
    y = selection %>% select(valence) %>% as.matrix(),
    alpha = 0,
    lambda = ridge_model$lambda.min)

min_ridge_model.coef <- min_ridge_model$beta

# Coefficients
model_coefs <- cbind(
  Full_OLS = coef(full_model),
  LASSO_min = c(
    min_lasso_model$a0,
    as.vector(min_lasso_model.coef)
  ),
  Ridge_min = c(
  min_ridge_model$a0,
  as.vector(min_ridge_model.coef)
  )
)
print("Table 6: Coefficients of the Features in our selected Model")
model_coefs

In [ ]:
summary(full_model)

Lasso and Ridge do not remove any features similar to the greedy selection methods, and the full model indicates that all variables are significant. Therefore, we will be keeping all of the predictive variables for our model.

In [ ]:
# Check for multicollinearity in the predictors
lasso_variables_vif <- 
   vif(lm(valence ~ popularity + acousticness + danceability + energy + instrumentalness +
         liveness + loudness + speechiness + tempo, data = selection))

lasso_variables_vif

While the highest VIFs correspond to the variables with the highest correlation (energy and loudness), there is no concerning presence of multicollinearity following James et al. (2013)'s method (only exceeding 5 or 10 poses problems). However, for our inference model, we will test to see if a model with energy & loudness may perform better or worse than without the variables to further prove that all variables are relevant.

### Inference

In [ ]:
#Dataset used to construct our models
print("Table 7: Dataset for Inference")
head(inference)

In [ ]:
# Model constructed using chosen covariates, without interaction
spotify_model <- lm(valence ~ ., data = inference)
spotify_model_summary = tidy(spotify_model, conf.int = TRUE) %>% mutate_if(is.numeric, round, 4)
print("Table 8: Coefficients for Full Model (Without Interactions)")
spotify_model_summary

In [ ]:
# Model constructed using covariates, without loudness and energy nor interactions
spotify_model_no_le <- lm(valence ~ popularity + acousticness + danceability + instrumentalness + liveness + speechiness + tempo, data = inference)
spotify_model_no_le_summary <- tidy(spotify_model_no_le, conf.int = TRUE) %>% mutate_if(is.numeric, round, 4)
print("Table 9: Coefficients for Model without Loudness and Energy (Without Interactions)")
spotify_model_no_le_summary

In [ ]:
print("Table 10: Summary Statistics for the Full Model (Without Interactions)")
glance(spotify_model)

In [ ]:
print("Table 11: Summary Statistics for the Reduced Model (Without Interactions)")
glance(spotify_model_no_le)

We see that the coefficient of determination is expectedly larger for the full model and that both perform better compared to the null model. However, we must use an ANOVA to assess whether the full model performs better than the nested one.

In [ ]:
print("Table 12: Summary Statistics from F-test for Reduced vs. Full Model")
anova(spotify_model_no_le, spotify_model)

We have evidence that the model with all variables performs significantly better than the model without the correlated variables loudness and energy. Thus, we conclude that the full model contains all relevant coefficients (popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness and tempo) and will use this to make our predictions.

As our full and LASSO models report different coefficients, we must still assess which model has better prediction performance.

In [ ]:
test_pred_full_OLS <- predict(spotify_model,
                              newx = inference %>% select(-valence) %>% as.matrix())

min_lasso_model_infer <- glmnet(
    x = inference %>% select(-valence) %>% as.matrix(),
    y = inference %>% select(valence) %>% as.matrix(),
    alpha = 1,
    lambda = lasso_model$lambda.min)

test_pred_LASSO_min <- predict(min_lasso_model_infer,
                               newx = inference %>% select(-valence) %>% as.matrix())

min_ridge_model_infer <- glmnet(
    x = inference %>% select(-valence) %>% as.matrix(),
    y = inference %>% select(valence) %>% as.matrix(),
    alpha = 0,
    lambda = ridge_model$lambda.min)

test_pred_ridge_min <- predict(min_ridge_model_infer,
                               newx = inference %>% select(-valence) %>% as.matrix())

R_MSE_models <- rbind(tibble(
  Model = "OLS Full Regression",
  R_MSE = rmse(
    preds = test_pred_full_OLS,
    actuals = inference$valence
  )
),
    tibble(
    Model = "LASSO Regression with minimum MSE",
    R_MSE = rmse(
      preds = test_pred_LASSO_min,
      actuals = inference$valence
    )
  ),
    tibble(
    Model = "Ridge Regression with minimum MSE",
    R_MSE = rmse(
      preds = test_pred_ridge_min,
      actuals = inference$valence
    )
  )
)
print("Table 13: Root mean squared errors of the Full, LASSO and Ridge models")
R_MSE_models

The OLS regression model has the lowest RMSE compared to the LASSO and Ridge models, thus we will use this to generate our predictions.

## Prediction

In [ ]:
#Dataset used for prediction
print("Table 14: Dataset for Prediction")
head(prediction_df)

In [ ]:
# Here, we get a vector of predicted values for our dataset using our model, 
predictions <- predict(spotify_model,
                       newdata = prediction_df %>% select(-valence) %>% data.frame())
pred_df <- tibble(
    predicted_value = predictions,
    actual_value = prediction_df$valence
)
print("Table 15: Predicted and Actual Values for Valence")
head(pred_df)
predictions_plot = pred_df %>%
    ggplot() +
    geom_point(aes(y = predicted_value, x = actual_value)) + 
    labs() +
    ggtitle("Figure 11. Predicted vs Actual Valence Value")
predictions_plot

Our model seems to loosely fit the data, with a similiar shape, however our model is not precise and seems to have high variation.

In [ ]:
pred_df = pred_df %>%
          mutate(residual = scale((actual_value - predicted_value)))
residuals_plot = pred_df %>%
                 ggplot() +
                 geom_point(aes(x = predicted_value, y = residual)) + 
                 labs(title = "Figure 12. Residuals-Fitted Value Plot", y = "Scaled Residual Value", x = "Predicted Value")
residuals_plot

Our residual plot has a clear linear trend, indicating homoscedasticity, and seems there may be variables affecting valence that our model is not accounting for, which might be the categorical variables we filtered out. 

In [ ]:
# Calculating the RMSE for our final model
inf_RMSE = R_MSE_models$R_MSE[2]
pred_RMSE = mean((pred_df$actual_value - pred_df$predicted_value)^2) %>% sqrt() 
cat("Training root mean squared error:", inf_RMSE, "\nTesting root mean squared error:", pred_RMSE)

Our inference RMSE (RMSE for the data we fit the model on) and prediction RMSE (for unknown data) are similiar, thus there is not any indication of overfitting or underfitting, and our model should generalize well.

## Discussion

### Findings
We determined that popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness and tempo may contribute to a Spotify song’s emotional valence. This was not expected as we had thought that some variables may be irrelevant for our analysis, however all coefficients were returned as significant and neither greedy selection method nor the regularization methods LASSO and Ridge identified removable features (Table 6). We can interpret this as each feature may contribute to the valence value to some degree and thus a combination of these linear variables would be best suited for making predictions on new music.

To select our inference model, we first compared the full model to a nested one without the highly correlated variables, loudness and energy, to see if the data had problems with multicollinearity, as we found that the variance inflation factors (VIF) for those variables were greater than the others. Loudness, a psychological measure of physical strength (amplitude), does logically relate to energy, which represents a perceptual measure of intensity and activity within a track; thus, we may expect energetic tracks that feel fast and noisy to also be typically louder. However, our F-test confirmed that the full model was significantly better than the reduced and therefore, the inclusion of both factors seems to fit the data better. This choice is also supported by James et al. ‘'s method (2013) which states that only VIFs exceeding 5 or 10 should pose problems for multicollinearity.

We also compared the root mean squared error (RMSE) of the ordinary least squares (OLS) model to the LASSO and Ridge models to assess predictive performance, where the full OLS model returned the lowest error on unseen data (Table 13). We can interpret this as the full OLS generating coefficients that are less biased when compared to the other models, reducing the amount of overfitting on the training set. From our estimates, we also observe that with all other variables held constant, valence will increase by 0.707 for every increase by 1 in danceability and similarly decrease by 0.119 for every increase by 1 in speechiness on average (Table 7). As some variables are between 0 and 1, we can interpolate to calculate the effects on valence for decimal changes in these features.

With the OLS model, we used a portion of our sample to generate predictions for unseen Spotify songs. We obtained a RMSE of approximately 0.198, thus our test predictions were generally about 0.20 off compared to the actual valence values. In other words, since valence is measured between 0 and 1, this means our model estimates have roughly 20% error on average. As our prediction (test) error is only 0.003 different from the inference (train) error, our model seems to generalize well to new data. However, this degree of error is still fairly large; a song’s valence could be predicted between 0.30 and 0.70 when its actual value is 0.50, thus leading us to assume its positivity or negativity when it is in fact neutral. Despite this, our other models produced similar RMSEs (Table 13), indicating that this level of predictive performance may be expected given the input variables used.

### Strengths

Our sample should be representative of all songs on Spotify as we do not expect the range of our input variables to change drastically (e.g. outside the bounds of human hearing) nor should there be significant deviations in valence values from one year to another or between regions of the world. Moreover, since we tested our inference model on a separate subsample, we assume that our model will perform reasonably well on other unseen Spotify data.

### Impacts and Implications

We made the decision to exclude categorical variables due to limits in computational power, however we cannot say for certain that these factors may not impact the valence. It is possible that genre could still be confounding; for example, certain feature values could affect the valence differentially if a song is classical or reggae given their different distributions (Figure 1). We also determined from Figure 12 that the data appears to be homoscedastic but biased, hence we may be missing another input variable in our analysis that may explain the diagonal pattern of the residuals. While valence is scaled between 0 and 1, the distribution is not precisely normal (Figure 5). Similarly, some input variables were skewed (ex. instrumentalness, liveness, loudness and speechiness). Therefore, we may receive better results from logarithmically transforming the feature values to normalize them prior to training the linear models, though these methods were not discussed in STAT 301.

Since the method that Spotify uses to quantify valence remains unknown to the public, we hope that our results provide clarity on how humans perceive music. It seems that acousticness, danceability, energy and tempo contribute to making songs more cheerful, whereas popularity, instrumentalness, liveness, loudness and speechiness may be associated with more depressing music (Table 7). As our impressions appear to be based on a variety of components, it is unlikely that we can design tracks which emphasize certain features to produce the “happiest track ever” or vice versa. However, if you listen to music that is more energetic and loud, our analysis supports that you may prefer listening to positively valenced songs. From this assumption, it may be reasonable to make suggestions to Spotify users based on their most frequently played tracks’ feature values, though further human research would be required to confirm.

### Future Questions

- How does valence affect the popularity of Spotify tracks and their longevity to remain on the top of rankings charts?
- Given the identified key contributors to valence, can these be attributed to making songs more or less likeable? Can we predict whether someone will enjoy a song based on these features?
- Is Spotify suggesting new songs to users based on their valence? What ethical implications may this measure have on the emotions and/or mood of listeners?



## References

Baldwin, C. L., & Lewis, B. A. (2017). Positive valence music restores executive control over 
sustained attention. PLOS ONE, 12(11). https://doi.org/10.1371/journal.pone.0186231 

Business of Apps. (2021, October 29). Spotify revenue and Usage Statistics (2021). Business of Apps. 
Retrieved November 4, 2021, from https://www.businessofapps.com/data/spotify-statistics/. 

Hamidani, Z. (2019, July 23). Spotify tracks DB. Kaggle. Retrieved November 4, 2021, from 
https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db. 

Roy, M., Peretz, I., & Rainville, P. (2008). Emotional valence contributes to music-induced 
analgesia. Pain, 134(1), 140–147. https://doi.org/10.1016/j.pain.2007.04.003 

Tajadura-Jiménez, A., Pantelidou, G., Rebacz, P., Västfjäll, D., & Tsakiris, M. (2011). I-space: 
The effects of emotional valence and source of music on interpersonal distance. PLOS ONE, 
6(10). https://doi.org/10.1371/journal.pone.0026083 